# 🐼 Modulo 1 - Introduccion a Pandas


- Deberás utilizar únicamente `pandas`y librerías estándar de Python para 

- resolver los distintos objetivos, este sera un proceso guiado para ayudarte a familiarizarte con la librería!

- Responde a TODAS las preguntas dentro de las celdas de ejecución 



## 💡 Contexto:
### El hospital "Stemdata" ha digitalizado su sistema de gestión de medicamentos. Te han entregado un archivo CSV
### con registros de la administración de medicamentos a pacientes, con las siguientes columnas:

- paciente_id: ID único del paciente
- nombre_medicamento: nombre del fármaco administrado
- dosis_mg: dosis suministrada (en miligramos)
- frecuencia_dia: veces al día que se administra el medicamento
- fecha_inicio: fecha de inicio del tratamiento
- fecha_fin: fecha de finalización del tratamiento (puede estar vacía si sigue activo)
- unidad: hospital, residencia o ambulatorio



## 📦 Ejercicio 1: Carga de datos
### Cargar los datos desde un archivo CSV

In [2]:
import pandas as pd

df = pd.read_csv('modulo_1.csv')
print("Datos cargados:\n", df.head())

Datos cargados:
    paciente_id nombre_medicamento  dosis_mg  frecuencia_dia fecha_inicio  \
0         1102         Loratadina     250.0               2   2023-12-22   
1         1435          Omeprazol     750.0               2   2023-11-29   
2         1860        Amoxicilina     750.0               3   2023-09-16   
3         1270        Amoxicilina     250.0               1   2023-06-10   
4         1106        Paracetamol    1000.0               1   2023-06-27   

       unidad   fecha_fin  
0    hospital  2024-02-11  
1  residencia  2023-12-12  
2  residencia  2023-12-28  
3    hospital  2023-07-04  
4  residencia  2023-10-31  


## 🧼 Ejercicio 2: Limpieza y validación
- Asegúrate de que no haya duplicados.
- Las fechas deben estar en formato datetime.
- Si fecha_fin está vacía, reemplázala por la fecha actual para calcular duración.
- Revisa si hay dosis nulas o negativas.

In [12]:
from datetime import datetime

# Eliminar duplicados
df.drop_duplicates(inplace=True)

# Convertir columnas de fecha a datetime
df['fecha_inicio'] = pd.to_datetime(df['fecha_inicio'])
df['fecha_fin'] = pd.to_datetime(df['fecha_fin'])

# Rellenar fecha_fin vacía con la fecha y hora actual
df['fecha_fin'] = df['fecha_fin'].fillna(pd.Timestamp(datetime.now()))

# Eliminar filas con dosis negativas o nulas
df = df[df['dosis_mg'] > 0]

print("Datos cargados:\n", df.head())


Datos cargados:
    paciente_id nombre_medicamento  dosis_mg  frecuencia_dia fecha_inicio  \
0         1102         Loratadina     250.0               2   2023-12-22   
1         1435          Omeprazol     750.0               2   2023-11-29   
2         1860        Amoxicilina     750.0               3   2023-09-16   
3         1270        Amoxicilina     250.0               1   2023-06-10   
4         1106        Paracetamol    1000.0               1   2023-06-27   

       unidad  fecha_fin  
0    hospital 2024-02-11  
1  residencia 2023-12-12  
2  residencia 2023-12-28  
3    hospital 2023-07-04  
4  residencia 2023-10-31  


### 📊 Ejercicio 3: Análisis general
- ¿Cuántos tratamientos únicos hay?
- ¿Cuántos pacientes únicos están registrados?
- ¿Cuáles son los 5 medicamentos más administrados?
- ¿Cuáles son las unidades con más pacientes tratados?

In [11]:

# Tratamientos únicos (paciente + medicamento)
tratamientos_unicos = df[['paciente_id', 'nombre_medicamento']].drop_duplicates().shape[0]
print("Tratamientos únicos:", tratamientos_unicos)

# Pacientes únicos
pacientes_unicos = df['paciente_id'].nunique()
print("Pacientes únicos:", pacientes_unicos)

# Medicamentos más administrados
medicamentos_top5 = df['nombre_medicamento'].value_counts().head(5)
print("Top 5 medicamentos más administrados:\n", medicamentos_top5)

# Unidades con más pacientes tratados
pacientes_por_unidad = df.groupby('unidad')['paciente_id'].nunique().sort_values(ascending=False)
print("Pacientes por unidad:\n", pacientes_por_unidad)

Tratamientos únicos: 278
Pacientes únicos: 239
Top 5 medicamentos más administrados:
 nombre_medicamento
Omeprazol      54
Paracetamol    53
Loratadina     49
Ibuprofeno     47
Amoxicilina    45
Name: count, dtype: int64
Pacientes por unidad:
 unidad
hospital       97
residencia     89
ambulatorio    82
Name: paciente_id, dtype: int64


### 🧠 Ejercicio 4: Cálculo de métricas clínicas
- Crea una columna "duracion_dias" con la diferencia entre fecha_fin y fecha_inicio.
- Calcula la "dosis_total_mg" por tratamiento (dosis x frecuencia_dia x duración_dias).
- ¿Cuál es la dosis total promedio por medicamento?
- ¿Qué paciente ha recibido la mayor dosis total acumulada?

In [13]:
# Calcular duración del tratamiento en días
df['duracion_dias'] = (df['fecha_fin'] - df['fecha_inicio']).dt.days

# Calcular dosis total por tratamiento
df['dosis_total_mg'] = df['dosis_mg'] * df['frecuencia_dia'] * df['duracion_dias']

# Dosis promedio por medicamento
dosis_promedio = df.groupby('nombre_medicamento')['dosis_total_mg'].mean()
print("Dosis promedio por medicamento:\n", dosis_promedio)

# Paciente con mayor dosis acumulada
paciente_max_dosis = df.groupby('paciente_id')['dosis_total_mg'].sum().idxmax()
print("Paciente con mayor dosis acumulada:", paciente_max_dosis)

Dosis promedio por medicamento:
 nombre_medicamento
Amoxicilina    311955.555556
Ibuprofeno     146175.531915
Loratadina     328418.367347
Metformina     240273.809524
Omeprazol      252861.111111
Paracetamol    189316.037736
Name: dosis_total_mg, dtype: float64
Paciente con mayor dosis acumulada: 1645


### 🛠️ Paso 5: Resolución de un problema
### La administración necesita saber:
- ¿Qué unidad tiene el mayor consumo total de medicamentos (en mg)?
- ¿Qué medicamento tiene mayor variabilidad en duración de tratamientos?
- ¿Qué tratamientos han durado más de 90 días?

In [14]:
# ¿Qué unidad tiene el mayor consumo total de medicamentos (en mg)?
consumo_por_unidad = df.groupby('unidad')['dosis_total_mg'].sum()
unidad_max_consumo = consumo_por_unidad.idxmax()
print("Unidad con mayor consumo total:", unidad_max_consumo)

# ¿Qué medicamento tiene mayor variabilidad en duración de tratamientos?
variabilidad_duracion = df.groupby('nombre_medicamento')['duracion_dias'].std()
medicamento_mayor_var = variabilidad_duracion.idxmax()
print("Medicamento con mayor variabilidad en duración:", medicamento_mayor_var)

# ¿Qué tratamientos han durado más de 90 días?
tratamientos_largos = df[df['duracion_dias'] > 90]
print("Tratamientos de más de 90 días:")
print(tratamientos_largos[['paciente_id', 'nombre_medicamento', 'duracion_dias']])

Unidad con mayor consumo total: hospital
Medicamento con mayor variabilidad en duración: Amoxicilina
Tratamientos de más de 90 días:
     paciente_id nombre_medicamento  duracion_dias
2           1860        Amoxicilina            103
4           1106        Paracetamol            126
6           1700         Metformina            127
7           1020          Omeprazol            167
10          1466         Loratadina            149
..           ...                ...            ...
291         1224          Omeprazol            138
295         1129         Ibuprofeno            131
297         1683          Omeprazol            780
298         1729        Paracetamol            879
299         1671        Paracetamol            110

[167 rows x 3 columns]
